In [1]:
import pandas as pd 
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np 
import joblib 
import requests



In [2]:

def create_embedding(text_list):
    # https://github.com/ollama/ollama/blob/main/docs/api.md#generate-embeddings
    r = requests.post("http://localhost:11434/api/embed", json={
        "model": "bge-m3",
        "input": text_list
    })

    embedding = r.json()["embeddings"] 
    return embedding


df = joblib.load('embeddings.joblib')




In [3]:
incoming_query = input("Ask a Question: ")
question_embedding = create_embedding([incoming_query])[0] 

similarities = cosine_similarity(np.vstack(df['embedding']), [question_embedding]).flatten()
top_results = 3
max_indx = similarities.argsort()[::-1][0:top_results]
#print(max_indx)
new_df = df.loc[max_indx] 


Ask a Question:  how to use list


In [6]:
# import requests
# import json
# from openai import OpenAI

# # Your data processing here
# prompt = f'''I put a youtube playlist for python course. Here are video subtitle chunks containing video title, video number, start time in seconds, end time in seconds, the text at that time:

# {new_df[["title", "number", "start", "end", "text"]].to_json(orient="records")}
# ---------------------------------
# "{incoming_query}"
# User asked this question related to the video chunks, you have to answer in a human way (dont mention the above format, its just for you) you have to give summarize answere of query and for reffrence tell where and how much content is taught in which video (in which video and at what timestamp) and guide the user to go to that particular video. If user asks unrelated question, tell him that you can only answer questions related to the course
# '''

# client = OpenAI(
#   base_url="https://openrouter.ai/api/v1",
#   api_key="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",
# )

# completion = client.chat.completions.create(
#   model="openai/gpt-4o",
#   messages=[
#     {
#       "role": "user",
#       "content": prompt
#     }
#   ],
#   max_tokens=3500  # Added this to stay within your 4000 token budget
# )

# print(completion.choices[0].message.content)

In [5]:
import requests
import json
import pandas as pd
from openai import OpenAI

# Filter to get only most relevant chunks
def filter_relevant_chunks(df, query, max_chars=2000):
    """Get most relevant chunks within character limit"""
    query_words = set(query.lower().split())
    df['relevance'] = df['text'].str.lower().apply(
        lambda x: len(query_words.intersection(set(x.split())))
    )
    
    # Sort by relevance and take chunks until we hit char limit
    sorted_df = df.nlargest(20, 'relevance')
    
    result_rows = []
    current_chars = 0
    for _, row in sorted_df.iterrows():
        row_json = json.dumps(row[["title", "number", "start", "end", "text"]].to_dict())
        if current_chars + len(row_json) < max_chars:
            result_rows.append(row)
            current_chars += len(row_json)
        else:
            break
    
    return pd.DataFrame(result_rows)

# Filter data first to reduce tokens
filtered_df = filter_relevant_chunks(new_df, incoming_query, max_chars=2000)

# Compact prompt
prompt = f'''You are a Python course assistant. Answer the user's question based on these video subtitles:

{filtered_df[["title", "number", "start", "end", "text"]].to_json(orient="records")}

User Question: "{incoming_query}"

Instructions:
1. Provide a clear answer to their question
2. Give video references: Video number, title, timestamp (convert seconds to MM:SS format)
3. Recommend which video(s) to watch

Only answer questions related to the Python course. If unrelated, politely say you can only help with course questions.'''
# 

client = OpenAI(
    base_url="https://openrouter.ai/api/v1",
    api_key="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",
)

completion = client.chat.completions.create(
    model="openai/gpt-4o",
    messages=[
        {
            "role": "user",
            "content": prompt
        }
    ],
    max_tokens=1500  # Reduced to fit within budget
)

print(completion.choices[0].message.content)

Prompt length: 1841 characters
To use a list in Python, you create a collection of items under one variable name. Lists can store multiple elements, which can be of different data types, such as integers, strings, or even other lists. Once you have created a list, you can iterate over its elements using loops.

Here’s a brief overview of using lists:
1. **Creating a List**: You define a list by placing items inside square brackets, separated by commas. For example:
   ```python
   numbers = [1, 2, 3, 4, 5]
   ```

2. **Iterating Over a List**: You can loop through the items in a list using a `for` loop:
   ```python
   for item in numbers:
       print(item)
   ```

Refer to the following videos for more information:
- **Video 11: "Learn Python Episode 11 Lists (Arrays)"** at timestamp **02:37** (157 seconds) where lists are introduced.
- **Video 19: "Learn Python Episode 19 Infinite Arguments"** at timestamp **01:34** (94 seconds) which further discusses using lists and iterating over